In [39]:
import pandas as pd

df1 = pd.read_csv("../data/1429_1.csv")
df2 = pd.read_csv("../data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv")
df3 = pd.read_csv("../data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

print("df1 columns:", df1.columns.tolist())
print("df2 columns:", df2.columns.tolist())
print("df3 columns:", df3.columns.tolist())


/var/folders/dt/6q8_7fhn65x7bhgb54cc6qbc0000gn/T/ipykernel_35486/2103305979.py:3: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("../data/1429_1.csv")


df1 columns: ['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer', 'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.userCity', 'reviews.userProvince', 'reviews.username']
df2 columns: ['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand', 'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber', 'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs']
df3 columns: ['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand', 'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber', 'reviews.date', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend',

In [40]:
import os

print(os.getcwd())


/Users/saramoshtaghi/Documents/Learning/GitHub/amazon-bert-classifier/notebook


In [41]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from utils.preprocessing import load_and_clean_data

df = load_and_clean_data(path="../data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
df.head()


,text,label
0,I order 3 of them and one of the item is bad q...,1
1,Bulk is always the less expensive way to go fo...,2
2,Well they are not Duracell but for the price i...,2
3,Seem to work as well as name brand batteries a...,2
4,These batteries are very long lasting the pric...,2


In [42]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), 
    test_size=0.2, random_state=42, stratify=df['label']
)


BERT Tokenizer

In [43]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [44]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

Pytorch Dataset

In [45]:
import torch

class AmazonReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = AmazonReviewDataset(train_encodings, train_labels)
val_dataset = AmazonReviewDataset(val_encodings, val_labels)


 Fine-tuning and deploying model

In [46]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Matrix

In [48]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}

In [49]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
from transformers import AutoTokenizer

# Assuming you're using a pre-trained BERT model
model_name = "bert-base-uncased"
num_labels = 2  # Change this to the number of labels in your dataset

# Load pre-trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Your training arguments
training_args = TrainingArguments(
    output_dir="./models/bert-amazon",
    evaluation_strategy="epoch",
    # ... other arguments ...
    metric_for_best_model="f1"
)

# Your trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/saramoshtaghi/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/dt/6q8_7fhn65x7bhgb54cc6qbc0000gn/T/ipykernel_35486/1295520961.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from sklearn.metrics import accuracy_score, f1_score
# import torch

# # 1. Load full dataset
# df = pd.read_csv("../data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
# df = df[['reviews.text', 'reviews.rating']].dropna()
# df.rename(columns={'reviews.text': 'text', 'reviews.rating': 'label'}, inplace=True)

# # 2. Map star ratings to sentiment labels
# def map_rating(r):
#     if r <= 2:
#         return 0  # negative
#     elif r == 3:
#         return 1  # neutral
#     else:
#         return 2  # positive

# df['label'] = df['label'].apply(map_rating)

# # 3. Use 10% of data for quick local training
# df_small = df.sample(frac=0.1, random_state=42)

# # 4. Train/Validation split
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     df_small['text'].tolist(), df_small['label'].tolist(),
#     test_size=0.2, random_state=42, stratify=df_small['label']
# )

# # 5. Tokenization
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# # 6. Dataset class
# class AmazonReviewDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels
#     def __len__(self):
#         return len(self.labels)
#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

# train_dataset = AmazonReviewDataset(train_encodings, train_labels)
# val_dataset = AmazonReviewDataset(val_encodings, val_labels)

# # 7. Load model
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# # 8. Metrics
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = np.argmax(logits, axis=1)
#     acc = accuracy_score(labels, preds)
#     f1 = f1_score(labels, preds, average='weighted')
#     return {"accuracy": acc, "f1": f1}

# # 9. Training config
# training_args = TrainingArguments(
#     output_dir="./models/bert-amazon-small",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=2,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1"
# )

# # 10. Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

# # 11. Train and evaluate
# trainer.train()
# trainer.evaluate()

# # 12. Save model
# model.save_pretrained("./models/bert-amazon-small")
# tokenizer.save_pretrained("./models/bert-amazon-small")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/saramoshtaghi/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/dt/6q8_7fhn65x7bhgb54cc6qbc0000gn/T/ipykernel_35486/4117457165.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.240706,0.941799,0.920326
2,0.296300,0.195659,0.947090,0.932231


('./models/bert-amazon-small/tokenizer_config.json',
 './models/bert-amazon-small/special_tokens_map.json',
 './models/bert-amazon-small/vocab.txt',
 './models/bert-amazon-small/added_tokens.json',
 './models/bert-amazon-small/tokenizer.json')

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()


/var/folders/dt/6q8_7fhn65x7bhgb54cc6qbc0000gn/T/ipykernel_35486/2167728092.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("./models/bert-amazon-finetuned")
tokenizer.save_pretrained("./models/bert-amazon-finetuned")

In [ ]:
from transformers import pipeline

finetuned_model = BertForSequenceClassification.from_pretrained("./models/bert-amazon-finetuned")
finetuned_tokenizer = AutoTokenizer.from_pretrained("./models/bert-amazon-finetuned")

classifier = pipeline("text-classification", model=finetuned_model, tokenizer=finetuned_tokenizer)

example = "The battery quality is poor and it stopped working after a week."
result = classifier(example)
print(result)


In [ ]:
from transformers import BertForSequenceClassification, AutoTokenizer

model = BertForSequenceClassification.from_pretrained("bert-amazon-finetuned")
tokenizer = AutoTokenizer.from_pretrained("bert-amazon-finetuned")

model.push_to_hub("bert-amazon-finetuned")
tokenizer.push_to_hub("bert-amazon-finetuned")


git init
git remote add origin https://github.com/saramoshtaghi/bert-amazon-finetuned
git add .
git commit -m "Initial commit: Fine-tuned BERT on Amazon reviews"
git push origin main